# **Lambda Code to transform Json to CSV in Kinesis Firehose**

In [ ]:
import json
import boto3
import base64

output = []

def lambda_handler(event, context):
    print(event)
    for record in event['records']:
        payload = json.loads(base64.b64decode(record['data']).decode('utf-8'))
        print('payload:', payload)
        
        output_payload=""
        for i in payload:
            output_payload=output_payload+str(payload[i])
            output_payload=output_payload+","
            	
        output_payload_clean=output_payload[:-1]+ "\n"
        
        output_payload_postprocessed = base64.b64encode(output_payload_clean.encode('utf-8'))
        
        output_record = {
            'recordId': record['recordId'],
            'result': 'Ok',
            'data': output_payload_postprocessed
        }
        output.append(output_record)

    print('Processed {} records.'.format(len(event['records'])))
    
    return {'records': output}

# **Data Producer**

In [ ]:
!pip install boto3

In [ ]:
aws_access_key=''
aws_secret_key=''
region=''

In [ ]:
import json
import datetime
import random
import boto3
session=boto3.Session(aws_access_key_id=aws_access_key,
                      aws_secret_access_key=aws_secret_key, region_name=region)

client = session.client('kinesis')

kinesis_stream="{}"


def getData(iotName, lowVal, highVal):
   data = {}
   data["iotName"] = iotName
   data["iotValue"] = random.randint(lowVal, highVal) 
   return data

temp=0;

while temp<100:
   rnd = random.random()
   if (rnd < 0.01):
      data = json.dumps(getData("DemoSensor", 100, 120))  
      client.put_record(StreamName=kinesis_stream, Data=data, PartitionKey="1")
      print('*************************** anomaly *********************** ' + data)
   else:
      data = json.dumps(getData("DemoSensor", 10, 20))  
      client.put_record(StreamName=kinesis_stream, Data=data, PartitionKey="1")
      print(data)
   temp=temp+1

# **Read the data from s3 **

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
.builder \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.3,com.amazonaws:aws-java-sdk:1.7.4") \
.config("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
.config("fs.s3a.access.key",aws_access_key ) \
.config("fs.s3a.secret.key", aws_secret_key) \
.getOrCreate()

In [ ]:
sdf = spark.read.option("header", "false").option("inferschema", "true").csv("{}")

In [ ]:
sdf.show()

In [ ]:
sdf.count()